# *Figures for ```some``` Experiment*
---
Generates analytics for 1 session 

> Example

> Rasters:
>> Aggression
>
>> Social Contact

> Aligned Averages:
>> Aggression
>
>> Social Contact

> Bar plots:
>> Aggression
>
>> Social Contact

## Setup

In [ ]:
#import everything and set dpi here

import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100 #set dpi for viewing figs within the notebook
import scipy.stats as stats

from matplotlib.patches import Rectangle

from scipy.stats import sem

## Load the infotable

In [ ]:
directory = r"C:/Users/njana/Desktop/Behavior/DualStim/Day1 Aug2223/" # include / at end
figdir = r"C:/Users/njana/Desktop/Behavior/DualStim/Figure/DualStim"

infofile = directory+"deets.csv"
infotable = pd.read_csv(infofile)

ctrl = []
chr2 = []
for i in range(infotable.shape[0]):
    if infotable["Type"][i] == "Control":
        ctrl += [i]  
    else:
        chr2 += [i]

infotable.sort_values("Type")

## Example Traces
Plot behavior panel with averages for ```SELECT ONE```

In [ ]:
i = 0 #select one

simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
ethoexportpath = directory+infotable.ethofile[i]+".txt"
ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
input1high = input1.loc[input1["Value"] == 1]
input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
input2high = input2.loc[input2["Value"] == 1]
input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
stimtimes = input2high["Recording time"].to_numpy()
stimtimes = np.asarray(stimtimes, dtype=float)

truncated = infotable.startf[i]
fps = 30
timetrunc = truncated/fps
stimtimes = stimtimes - timetrunc
    
stimstartidx = np.around(stimtimes*fps).astype("int")

stimdur = infotable.stimdur[i] #in seconds
stimidxdur = stimdur*fps
stimendidx = stimstartidx + stimidxdur

stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
BlindID = str(infotable.BlindID[i])
MouseID = str(infotable.MouseID[i])
Type = str(infotable.Type[i])        
day = str(infotable.Day[i])
trial = str(infotable.Trial[i])
time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
avgtime = np.arange(0,stimdur*3*fps)/fps

    
colors = ["red","brown","orange","purple","green","turquoise","black","sandybrown"]
sns.set_style("ticks")

#plt.subplots(9,2,gridspec_kw={'width_ratios': [time.shape[0],stimdur*fps*3]})
plt.subplots(9,2,gridspec_kw={'width_ratios': [5,1]}) #15,1
#plt.subplots_adjust(hspace=0)
plt.subplots_adjust(wspace=0)


plt.subplot(9,2,1)
plt.plot(time,stim, color = 'blue')
plt.ylabel("Stimulus", color = 'blue', rotation = 'horizontal', ha = 'left', va = "center", fontsize = 16, labelpad = 75, fontname = 'Arial')
plt.gca().set_xticks([]) 
plt.gca().set_yticks([])
    
plt.subplot(9,2,2)
plt.plot(avgtime,stim[stimstartidx[0]-(stimdur*fps):stimstartidx[0]+(stimdur*fps*2)], color = 'blue')
plt.gca().set_xticks([]) 
plt.gca().set_yticks([])
    
counter = 0
plotnum = 3

behav_names = ["Aggression", "Mounting","Dominate", 'Following', "Social\nContact", "Asocial", "Passive","Rest\nMove"]

for k in range(8):
        
        plt.subplot(9,2,plotnum)
        plt.plot(time,behavdata[behavdata.columns[counter]],  color = colors[k], alpha = 0.3)
        counter += 1
        
        plt.plot(time,behavdata[behavdata.columns[counter]], color = colors[k], alpha = 1)
        
        plt.ylabel(behav_names[k], color = colors[k], rotation = 'horizontal', ha = 'left', va = "center", fontsize = 16, labelpad = 75, fontname = 'Arial')
        
        if plotnum < 17:
            plt.gca().set_xticks([])
        else:
            plt.gca().set_xticks(np.arange(min(time), max(time)+1, 300), labels = np.arange(min(time), max(time)+1, 300).astype(int), size = 10, fontname = 'Arial')
        plt.gca().set_yticks([])
        sns.despine(bottom = True, left = True)
        
        plotnum += 1
        
        plt.subplot(9,2,plotnum)
        
        # calculate the average data
        behav = behavdata[behavdata.columns[counter]].to_numpy()
        window_behav = []
        for ele in stimstartidx:
            window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
        window_behav = np.array(window_behav)
        avg_behav = np.average(window_behav, axis = 0)
    
        # plot the average data
        plt.plot(avgtime,avg_behav, color = colors[k], alpha = 1)
        plt.ylim(bottom = 0)
        plt.gca().fill_between(avgtime, np.ones(2700),  where=stim[stimstartidx[0]-(stimdur*fps):stimstartidx[0]+(stimdur*fps*2)] > 0, facecolor='blue', alpha=.2)
        if plotnum < 17:
            plt.gca().set_xticks([])
        else:
            plt.gca().set_xticks([0,30,60,90], labels = [-30,0,30,60], size = 10, fontname = 'Arial')
        plt.gca().set_yticks([])
        sns.despine(bottom = True, left = True)        
            
        counter += 1   
        plotnum += 1
    
#plt.gcf().set_size_inches(16,8)
plt.gcf().set_size_inches(7.56,4.2)
plt.savefig(figdir+"_example.svg")#, dpi = 600)       

## Rasters
### Aggression

In [ ]:
# two blocks of stims
ctrl = []
chr2 = []
for i in range(infotable.shape[0]):
    if infotable["Type"][i] == "Control":
        ctrl += [i]  
    else:
        chr2 += [i]

agg = []
aggavg = []


#for i in range(1):
for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
    BlindID = str(infotable.BlindID[i])
    MouseID = str(infotable.MouseID[i])
    Type = str(infotable.Type[i])        
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    # collect behavior data
    behav = behavdata[behavdata.columns[1]].to_numpy() # 1 - aggression 9 - social contact
    agg += [behav[stimstartidx[0]-stimidxdur:stimendidx[-1]+stimidxdur]]
    
    # collect average data
    window_behav = []
    for ele in stimstartidx:
            if ele > stimdur*fps and ele < (behavdata[behavdata.columns[0]].shape[0] - stimdur*fps*2):
                window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
    window_behav = np.array(window_behav)
    aggavg+= [np.average(window_behav, axis = 0)]
    
stim = np.zeros(27000)

stimstarts = np.arange(900, 27000, 2700)


for ele in stimstarts:
    stim[ele:ele+900] = 1
    
sns.set_theme(style="ticks", palette = 'deep')

plt.subplots(8,2,gridspec_kw={'width_ratios': [4,1]}) #15,1
plt.subplots_adjust(hspace=0)
plt.subplots_adjust(wspace=0)

plt.subplot(8,2,1)
plt.plot(stim, c = 'b')
#plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000), stim,  where=stim > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)


plt.subplot(8,2,2)
plt.plot(stim[:2700], c = 'b')
#plt.gca().fill_between(np.arange(0,2700,1), np.zeros(2700), stim[:2700],  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)

mname = ["R1","R2","R3"]

for i, ele in enumerate(chr2):
    #i = i*2
    plt.subplot(8,2, i*2+3)
    plt.plot(agg[ele], c = 'r')
    plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000), stim*2,  where=stim > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,1.2)
    plt.yticks([])
    plt.xticks([])
    plt.ylabel(mname[i], rotation = 'horizontal', va = 'center')
    
for i, ele in enumerate(chr2):
    i = i*2
    plt.subplot(8,2, i+4)
    plt.plot(aggavg[ele], c = 'r')
    plt.gca().fill_between(np.arange(0,2700,1), np.ones(2700),  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,0.5)
    plt.yticks([])
    plt.xticks([])
    
plt.subplot(8,2,9)
plt.plot(stim, c = 'b')
#plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000), stim,  where=stim > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)


plt.subplot(8,2,10)
plt.plot(stim[:2700], c = 'b')
#plt.gca().fill_between(np.arange(0,2700,1), np.zeros(2700), stim[:2700],  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)

mname = ["R4","R5","R6"]

for i, ele in enumerate(ctrl):
    #i = i*2
    plt.subplot(8,2, i*2+11)
    plt.plot(agg[ele], c = 'grey')
    plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000),  stim*2, where=stim > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,1.2)
    plt.yticks([])
    plt.xticks([])
    plt.ylabel(mname[i], rotation = 'horizontal', va = 'center')

plt.xticks(ticks = np.append(900, np.arange(900, 27000, 2700)), labels = np.arange(0,11,1)*30)

    
for i, ele in enumerate(ctrl):
    i = i*2
    
    plt.subplot(8,2,i+12)
    plt.plot(aggavg[ele], c = 'grey')
    plt.gca().fill_between(np.arange(0,2700,1), np.ones(2700),  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,0.4)
    plt.xticks([])
    plt.yticks([])
    
    
plt.xticks([0, 900, 1800, 2700],[-30, 0, 30, 60])
sns.despine(bottom = 'True', left = 'True')

plt.gcf().set_size_inches(6,5)

plt.savefig(figdir+"_agg_aligned.svg")#, dpi = 600)    

## Social Contact

In [ ]:
ctrl = []
chr2 = []
for i in range(infotable.shape[0]):
    if infotable["Type"][i] == "Control":
        ctrl += [i]  
    else:
        chr2 += [i]

agg = []
aggavg = []


#for i in range(1):
for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
    BlindID = str(infotable.BlindID[i])
    MouseID = str(infotable.MouseID[i])
    Type = str(infotable.Type[i])        
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    # collect behavior data
    behav = behavdata[behavdata.columns[9]].to_numpy() # 1 - aggression 9 - social contact
    agg += [behav[stimstartidx[0]-stimidxdur:stimendidx[-1]+stimidxdur]]
    
    # collect average data
    window_behav = []
    for ele in stimstartidx:
            if ele > stimdur*fps and ele < (behavdata[behavdata.columns[0]].shape[0] - stimdur*fps*2):
                window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
    window_behav = np.array(window_behav)
    aggavg+= [np.average(window_behav, axis = 0)]
    
stim = np.zeros(27000)

stimstarts = np.arange(900, 27000, 2700)


for ele in stimstarts:
    stim[ele:ele+900] = 1
    
sns.set_theme(style="ticks", palette = 'deep')

plt.subplots(8,2,gridspec_kw={'width_ratios': [4,1]}) #15,1
plt.subplots_adjust(hspace=0)
plt.subplots_adjust(wspace=0)

plt.subplot(8,2,1)
plt.plot(stim, c = 'b')
#plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000), stim,  where=stim > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)


plt.subplot(8,2,2)
plt.plot(stim[:2700], c = 'b')
#plt.gca().fill_between(np.arange(0,2700,1), np.zeros(2700), stim[:2700],  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)

mname = ["R1","R2","R3"]

for i, ele in enumerate(chr2):
    #i = i*2
    plt.subplot(8,2, i*2+3)
    plt.plot(agg[ele], c = 'g')
    plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000), stim*2,  where=stim > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,1.2)
    plt.yticks([])
    plt.xticks([])
    plt.ylabel(mname[i], va = 'center', rotation = 'horizontal')
    
for i, ele in enumerate(chr2):
    i = i*2
    plt.subplot(8,2, i+4)
    plt.plot(aggavg[ele], c = 'g')
    plt.gca().fill_between(np.arange(0,2700,1), np.ones(2700),  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,0.5)
    plt.yticks([])
    plt.xticks([])
    
plt.subplot(8,2,9)
plt.plot(stim, c = 'b')
#plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000), stim,  where=stim > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)


plt.subplot(8,2,10)
plt.plot(stim[:2700], c = 'b')
#plt.gca().fill_between(np.arange(0,2700,1), np.zeros(2700), stim[:2700],  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
plt.yticks([])
plt.xticks([])
plt.ylim(-0.2,2)

for i, ele in enumerate(ctrl):
    #i = i*2
    plt.subplot(8,2, i*2+11)
    plt.plot(agg[ele], c = 'grey')
    plt.gca().fill_between(np.arange(0,27000,1), np.zeros(27000),  stim*2, where=stim > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,1.2)
    plt.yticks([])
    plt.xticks([])
    plt.ylabel(mname[i], va = 'center', rotation = 'horizontal')

plt.xticks(ticks = np.append(900, np.arange(900, 27000, 2700)), labels = np.arange(0,11,1)*30, fontname = 'Arial')

    
for i, ele in enumerate(ctrl):
    i = i*2
    
    plt.subplot(8,2,i+12)
    plt.plot(aggavg[ele], c = 'grey')
    plt.gca().fill_between(np.arange(0,2700,1), np.ones(2700),  where=stim[:2700] > 0, facecolor='blue', alpha=.2)
    plt.ylim(0,0.4)
    plt.xticks([])
    plt.yticks([])
    
    
plt.xticks([0, 900, 1800, 2700],[-30, 0, 30, 60], fontname = 'Arial')
sns.despine(bottom = 'True', left = 'True')

plt.gcf().set_size_inches(6,5)

plt.savefig(figdir+"_socon_aligned.svg")#, dpi = 600)    

## Aligned Average: Aggression

In [ ]:
''' Averaged agg traces '''

pre = []
dur = []
post = []
preavg = []
duravg = []
postavg = []

arrs = []

#for i in range(1):
for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
    BlindID = str(infotable.BlindID[i])
    MouseID = str(infotable.MouseID[i])
    Type = str(infotable.Type[i])        
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    # calculate the average data
    behav = behavdata[behavdata.columns[1]].to_numpy() # 1 - aggression 9 - social contact
    window_behav = []
    for ele in stimstartidx:
            if ele > stimdur*fps and ele < (behavdata[behavdata.columns[0]].shape[0] - stimdur*fps*2):
                window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
    window_behav = np.array(window_behav)
    avg_behav = np.average(window_behav, axis = 0)
    sum_behav = window_behav.sum(axis = 0)
    
    pre += [sum_behav[:int(sum_behav.shape[0]/3)].sum()/30]
    dur += [sum_behav[int(sum_behav.shape[0]/3):int(sum_behav.shape[0]*2/3)].sum()/30]
    post += [sum_behav[int(sum_behav.shape[0]*2/3):].sum()/30]

    preavg += [avg_behav[:int(avg_behav.shape[0]/3)].sum()/30]
    duravg += [avg_behav[int(avg_behav.shape[0]/3):int(avg_behav.shape[0]*2/3)].sum()/30]
    postavg += [avg_behav[int(avg_behav.shape[0]*2/3):].sum()/30]
    
    arrs += [avg_behav]
    
sns.set_style("white")
ctrl = []
chr2 = []
for i in range(len(arrs)):
    if infotable["Type"][i] == "Control":
        ctrl += [arrs[i]]  
    else:
        chr2 += [arrs[i]]

plt.plot(avgtime, np.mean(ctrl, axis = 0), c = 'grey', alpha = 1)
plt.fill_between(avgtime, np.mean(ctrl, axis = 0) - sem(ctrl, axis = 0), np.mean(ctrl, axis = 0) + sem(ctrl, axis = 0), color = 'grey', alpha= 0.2)
plt.ylabel("Average Aggression", fontname = 'Arial', size = 16)
plt.plot(avgtime, np.mean(chr2, axis = 0), c = 'r')
plt.fill_between(avgtime,  np.mean(chr2, axis = 0) - sem(chr2, axis = 0),  np.mean(chr2, axis = 0) + sem(chr2, axis = 0), color = 'r', alpha= 0.2)
plt.xlabel("Time Aligned to Stimulus (s)", fontname = 'Arial', size = 16)
maskstim = np.zeros(2700)
maskstim[900:1800] = 1
plt.gca().fill_between(avgtime, np.zeros(2700),  maskstim*0.6, facecolor='blue', alpha=.1)
plt.ylim(0,0.27)
plt.yticks(fontname = 'arial', fontsize = 12)

plt.gca().set_xticks([0,30,60,90],[-30,0,30,60], fontname = 'Arial', size = 12)
sns.despine()

plt.gcf().set_size_inches(4.75,5.25)
plt.savefig(figdir+"_agg_probs.svg")#, dpi = 600)    

## Aligned Average: Social Contact

In [ ]:
''' Averaged socon traces '''

pre = []
dur = []
post = []
preavg = []
duravg = []
postavg = []

arrs = []

#for i in range(1):
for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
    BlindID = str(infotable.BlindID[i])
    MouseID = str(infotable.MouseID[i])
    Type = str(infotable.Type[i])        
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    # calculate the average data
    behav = behavdata[behavdata.columns[9]].to_numpy() # 1 - aggression 9 - social contact
    window_behav = []
    for ele in stimstartidx:
            if ele > stimdur*fps and ele < (behavdata[behavdata.columns[0]].shape[0] - stimdur*fps*2):
                window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
    window_behav = np.array(window_behav)
    avg_behav = np.average(window_behav, axis = 0)
    sum_behav = window_behav.sum(axis = 0)
    
    pre += [sum_behav[:int(sum_behav.shape[0]/3)].sum()/30]
    dur += [sum_behav[int(sum_behav.shape[0]/3):int(sum_behav.shape[0]*2/3)].sum()/30]
    post += [sum_behav[int(sum_behav.shape[0]*2/3):].sum()/30]

    preavg += [avg_behav[:int(avg_behav.shape[0]/3)].sum()/30]
    duravg += [avg_behav[int(avg_behav.shape[0]/3):int(avg_behav.shape[0]*2/3)].sum()/30]
    postavg += [avg_behav[int(avg_behav.shape[0]*2/3):].sum()/30]
    
    arrs += [avg_behav]
    
sns.set_style("white")
ctrl = []
chr2 = []
for i in range(len(arrs)):
    if infotable["Type"][i] == "Control":
        ctrl += [arrs[i]]  
    else:
        chr2 += [arrs[i]]

plt.plot(avgtime, np.mean(ctrl, axis = 0), c = 'grey', alpha = 1)
plt.fill_between(avgtime, np.mean(ctrl, axis = 0) - np.std(ctrl, axis = 0), np.mean(ctrl, axis = 0) + sem(ctrl, axis = 0), color = 'grey', alpha= 0.2)
plt.ylabel("Averag Social Contact", fontname = 'Arial', size = 16)
plt.plot(avgtime, np.mean(chr2, axis = 0), c = 'green')
plt.fill_between(avgtime,  np.mean(chr2, axis = 0) - np.std(chr2, axis = 0),  np.mean(chr2, axis = 0) + sem(chr2, axis = 0), color = 'green', alpha= 0.2)
plt.xlabel("Time Aligned to Stimulus (s)", fontname = 'Arial', size = 16)
maskstim = np.zeros(2700)
maskstim[900:1800] = 1
plt.gca().fill_between(avgtime, np.zeros(2700),  maskstim*0.6, facecolor='blue', alpha=.1)
plt.ylim(0,0.58)
plt.yticks(fontname = 'arial', fontsize = 12)

plt.gca().set_xticks([0,30,60,90],[-30,0,30,60], fontname = 'Arial', size = 12)
sns.despine()

plt.gcf().set_size_inches(4.75,5.25)
plt.savefig(figdir+"_socon_probs.svg")#, dpi = 600)    

## Barplots: Aggression

In [ ]:
''' quantified Aggression ''' 


pre = []
dur = []
post = []
preavg = []
duravg = []
postavg = []

arrs = []
rawarrs = []

#for i in range(1):
for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
    BlindID = str(infotable.BlindID[i])
    MouseID = str(infotable.MouseID[i])
    Type = str(infotable.Type[i])        
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    # calculate the average data
    behav = behavdata[behavdata.columns[1]].to_numpy() # 1 - aggression 9 - social contact
    window_behav = []
    for ele in stimstartidx:
            if ele > stimdur*fps and ele < (behavdata[behavdata.columns[0]].shape[0] - stimdur*fps*2):
                window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
    rawarrs += [window_behav]
    window_behav = np.array(window_behav)
    avg_behav = np.average(window_behav, axis = 0)
    sum_behav = window_behav.sum(axis = 0)
    
    pre += [sum_behav[:int(sum_behav.shape[0]/3)].sum()/300]
    dur += [sum_behav[int(sum_behav.shape[0]/3):int(sum_behav.shape[0]*2/3)].sum()/300]
    post += [sum_behav[int(sum_behav.shape[0]*2/3):].sum()/300]

    preavg += [avg_behav[:int(avg_behav.shape[0]/3)].sum()/300]
    duravg += [avg_behav[int(avg_behav.shape[0]/3):int(avg_behav.shape[0]*2/3)].sum()/300]
    postavg += [avg_behav[int(avg_behav.shape[0]*2/3):].sum()/300]
    
    arrs += [avg_behav]
    
ctrl = []
chr2 = []
for i in range(infotable.shape[0]):
    if infotable["Type"][i] == "Control":
        ctrl += [i]  
    else:
        chr2 += [i]

prec = []
durc = []
postc = []

pree = []
dure = []
poste = []

for ele in ctrl:
    prec += [pre[ele]]
    durc += [dur[ele]]
    postc += [post[ele]]
    
for ele in chr2:
    pree += [pre[ele]]
    dure += [dur[ele]]
    poste += [post[ele]]
    
# aggression barplots

sns.set_theme(style="white", palette = 'deep')
plt.subplots_adjust(wspace=0.4)

plt.subplot(1,2,1)
ctrl = np.array([prec,durc])#,postc])
plt.bar(x = [0,1.5], height = np.mean(ctrl, axis = 1), color = 'grey')
plt.errorbar(x = [0,1.5], y = np.mean(ctrl, axis = 1), yerr = sem(ctrl, axis = 1), linestyle='', color = 'black', capsize = 10)
plt.scatter(np.ones(3)*-0.1,prec, c = 'black')
plt.scatter(np.ones(3)*1.4,durc, c = 'black')
for i in range(3):
    plt.plot([-0.1,1.4], [prec[i],durc[i]], color = 'black', linewidth = 0.5)

plt.xticks([0,1.5],["Pre-stim","Stimulus"], fontname = 'Arial', size = 14)
plt.ylabel("Avg time engaged in Aggression/30s", fontname = 'Arial', size = 16)

plt.ylim(0,2.5)
plt.hlines(2.2,0,1.5, color = 'black')
plt.text(0.75,2.3,"p = "+str(np.around(stats.ttest_rel(prec,durc)[1],3)), horizontalalignment = 'center', style = 'italic', size = 14, fontname = 'Arial')

plt.subplot(1,2,2)
chr2 = np.array([pree,dure])#,poste])
plt.bar(x = [0,1.5], height = np.mean(chr2, axis = 1), color = 'r', width = 1)
plt.errorbar(x = [0,1.5], y = np.mean(chr2, axis = 1), yerr = sem(chr2, axis = 1), linestyle='', color = 'black', capsize = 10)
plt.scatter(np.ones(3)*-0.1,pree, c = 'black')
plt.scatter(np.ones(3)*1.4,dure, c = 'black')
plt.xticks([0,1.5],["Pre-stim","Stimulus"], fontname = 'Arial', size = 14)
for i in range(3):
    plt.plot([-0.1,1.4], [pree[i],dure[i]], color = 'black', linewidth = 0.5)

plt.ylim(0,2.5)
plt.hlines(2.2,0,1.5, color = 'black')
plt.text(0.75,2.3,"p = "+str(np.around(stats.ttest_rel(pree,dure)[1],3)), horizontalalignment = 'center', style = 'italic', size = 14, fontname = 'Arial')

sns.despine()
plt.gcf().set_size_inches(4.75,5.25)
plt.savefig(figdir+"_agg_bars.svg")#, dpi = 600)       

## Barplots: Social Contact

In [ ]:
''' quantified social contact ''' 


pre = []
dur = []
post = []
preavg = []
duravg = []
postavg = []

arrs = []
rawarrs = []

#for i in range(1):
for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
    BlindID = str(infotable.BlindID[i])
    MouseID = str(infotable.MouseID[i])
    Type = str(infotable.Type[i])        
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    # calculate the average data
    behav = behavdata[behavdata.columns[9]].to_numpy() # 1 - aggression 9 - social contact
    window_behav = []
    for ele in stimstartidx:
            if ele > stimdur*fps and ele < (behavdata[behavdata.columns[0]].shape[0] - stimdur*fps*2):
                window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
    rawarrs += [window_behav]
    window_behav = np.array(window_behav)
    avg_behav = np.average(window_behav, axis = 0)
    sum_behav = window_behav.sum(axis = 0)
    
    
    #total
    pre += [sum_behav[:int(sum_behav.shape[0]/3)].sum()/300]
    dur += [sum_behav[int(sum_behav.shape[0]/3):int(sum_behav.shape[0]*2/3)].sum()/300]
    post += [sum_behav[int(sum_behav.shape[0]*2/3):].sum()/300]
    
    '''
    #total[20]
    pre += [sum_behav[300:int(sum_behav.shape[0]/3)].sum()/300]
    dur += [sum_behav[int(sum_behav.shape[0]/3)+300:int(sum_behav.shape[0]*2/3)].sum()/300]
    post += [sum_behav[int(sum_behav.shape[0]*2/3):].sum()/300]    
    '''
    #average
    preavg += [avg_behav[:int(avg_behav.shape[0]/3)].sum()/30]
    duravg += [avg_behav[int(avg_behav.shape[0]/3):int(avg_behav.shape[0]*2/3)].sum()/30]
    postavg += [avg_behav[int(avg_behav.shape[0]*2/3):].sum()/30]
    
    arrs += [avg_behav]
    
ctrl = []
chr2 = []
for i in range(infotable.shape[0]):
    if infotable["Type"][i] == "Control":
        ctrl += [i]  
    else:
        chr2 += [i]

prec = []
durc = []
postc = []

pree = []
dure = []
poste = []

for ele in ctrl:
    prec += [pre[ele]]
    durc += [dur[ele]]
    postc += [post[ele]]
    
for ele in chr2:
    pree += [pre[ele]]
    dure += [dur[ele]]
    poste += [post[ele]]

# social contact barplots

sns.set_theme(style="white", palette = 'deep')
plt.subplots_adjust(wspace=0.4)
plt.subplot(1,2,1)
ctrl = np.array([prec,durc])#,postc])
plt.bar(x = [0,1.5], height = np.mean(ctrl, axis = 1), color = 'grey')
plt.errorbar(x = [0,1.5], y = np.mean(ctrl, axis = 1), yerr = sem(ctrl, axis = 1), linestyle='', color = 'black', capsize = 10)
plt.scatter(np.ones(3)*-0.1,prec, c = 'black')
plt.scatter(np.ones(3)*1.4,durc, c = 'black')
for i in range(3):
    plt.plot([-0.1,1.4], [prec[i],durc[i]], color = 'black', linewidth = 0.5)

plt.xticks([0,1.5],["Pre-stim","Stimulus"], fontname = 'Arial', size = 14)
plt.ylabel("Avg Time spent in Social Contact/30s", fontname = 'Arial', size = 16)

plt.ylim(0,4.2)
plt.yticks(fontname = 'Arial', size = 12)
plt.hlines(4,0,1.5, color = 'black')
plt.text(0.75,4.1,"p = "+str(np.around(stats.ttest_rel(prec,durc)[1],3)), horizontalalignment = 'center', style = 'italic', size = 14, fontname = 'Arial')

plt.subplot(1,2,2)
chr2 = np.array([pree,dure])#,poste])
plt.bar(x = [0,1.5], height = np.mean(chr2, axis = 1), color = 'g', width = 1)
plt.errorbar(x = [0,1.5], y = np.mean(chr2, axis = 1), yerr = sem(chr2, axis = 1), linestyle='', color = 'black', capsize = 10)
plt.scatter(np.ones(3)*-0.1,pree, c = 'black')
plt.scatter(np.ones(3)*1.4,dure, c = 'black')
plt.xticks([0,1.5],["Pre-stim","Stimulus"], fontname = 'Arial', size = 14)
for i in range(3):
    plt.plot([-0.1,1.4], [pree[i],dure[i]], color = 'black', linewidth = 0.5)

plt.ylim(0,4.2)
plt.yticks(fontname = 'Arial', size = 12)
plt.hlines(4,0,1.5, color = 'black')
plt.text(0.75,4.1,"p = "+str(np.around(stats.ttest_rel(pree,dure)[1],3)), horizontalalignment = 'center', style = 'italic', size = 14, fontname = 'Arial')

sns.despine()
plt.gcf().set_size_inches(4.75,5.25)

plt.savefig(figdir+"_socon_bars.svg")#, dpi = 600)     

---
---